In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 4.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook


In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores),  np.std(scores)

In [15]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [16]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [17]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [23]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [30]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [31]:
def get_name_feat(key):
  return 'feat_' + key 

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [32]:
df[df['feat_athlete'].isnull()].shape

(18272, 526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [39]:
{k:v for k, v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
#@title Default title text
df['feat_brand_cat'] = df['feat_brand'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_color_cat'] = df['feat_color'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_material_cat'] = df['feat_material'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_sport_cat'] = df['feat_sport'].map(lambda x: str(x).lower()).factorize()[0]
df['feat_style_cat'] = df['feat_style'].map(lambda x: str(x).lower()).factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].map(lambda x: str(x).lower()).factorize()[0]


In [45]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand != df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [46]:
df[df.brand != df.feat_brand].shape

(9434, 531)

In [48]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.30429943797971, 4.252540751007453)

In [69]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_clothing category',
 'feat_certifications and listings',
 'feat_shoe category',
 'feat_catalog',
 'feat_recommended location',
 'feat_multi pack indicator',
 'feat_location - country',
 'feat_fabrication',
 'feat_location - city/state',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_use_cat',
 'feat_capacity_cat',
 'feat_is weather-resistant_cat',
 'feat_manufacturer sku_cat',
 'feat_material type_cat',
 'feat_country of origin_cat',
 'feat_sock style_cat',
 'feat_power type_cat',
 'feat_polarized?_cat',
 'feat_uv rating_cat',
 'feat_dial markers_cat',
 'feat_batteries included?_cat',
 'feat_closure_cat',
 'feat_heat zones_cat',
 'feat_fabric care instructions_cat',
 'feat_is lined_cat',
 'feat_autographed_cat',
 'feat_adjustable_cat',
 'feat_lens color_cat',
 'feat_country of origin assembly:_cat',
 'feat_flame resistant_cat',
 'feat_s

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat', 'feat_shape_cat', 'feat_metal type_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [83]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)
print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names = feats)

(-57.31903191953444, 4.285267684370464)


Weight,Feature
0.2576 ± 0.0075,brand_cat
0.1047 ± 0.0162,feat_material_cat
0.0249 ± 0.0040,feat_gender_cat
0.0178 ± 0.0007,feat_brand_cat
0.0128 ± 0.0004,feat_shape_cat
0.0083 ± 0.0014,feat_metal type_cat
0.0034 ± 0.0008,feat_style_cat
0.0001 ± 0.0000,feat_sport_cat


In [84]:
df['brand'].value_counts(normalize = True)

nike               0.097210
puma               0.033315
ralph lauren       0.028775
vans               0.021116
new balance        0.020295
                     ...   
rance              0.000055
ride snowboards    0.000055
unbrand            0.000055
nba                0.000055
xtratuf            0.000055
Name: brand, Length: 1732, dtype: float64

In [62]:
df[df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [67]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
infant                 4
mens                   4
toddler                4
boys'                  3
youth                  2
women ,�� unisex       2
women                  2
men||women             2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [101]:
ls

ColabDriveGithub.ipynb  HelloGithub.ipynb  matrix_one/
data/                   LICENSE            README.md



*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@74136d81922c.(none)')


In [0]:
!git config --global user.email "ewelina.swend@gmail.com"
!git config --global user.name "Ewelina"

In [90]:
!git commit -m "ML project Day 5"

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
	modified:   matrix_one/day3.ipynb

Untracked files:
	ColabDriveGithub.ipynb

no changes added to commit


In [91]:
!git push -u origin master

Branch 'master' set up to track remote branch 'master' from 'origin'.
Everything up-to-date
